<a href="https://colab.research.google.com/github/N10dx/Bootcamp-Project-1-Python/blob/main/Improved_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD, Adadelta, RMSprop, Adagrad, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd




In [4]:
data = pd.read_csv('diabetes.csv')
X = data.iloc[:,0:8]
Y = data.iloc[:,8]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:

def create_my_model(optimizer='adam', dropout_rate=0.0, use_batch_norm=False, neurons=12):
    mymodel = Sequential()
    mymodel.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    if use_batch_norm:
        mymodel.add(BatchNormalization())
    if dropout_rate > 0:
        mymodel.add(Dropout(dropout_rate))

    mymodel.add(Dense(1, activation='sigmoid'))
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel


In [6]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (614, 8)
y_train shape: (614,)


In [7]:

optimizers = ['SGD', 'Adadelta', 'RMSprop', 'Adagrad', 'Adam']
dropout_rates = [0.0, 0.2, 0.4]
epochs = [200, 300]
use_batch_norms = [True, False]
batchSize = [50, 100]
neurons = [8, 12, 16]

param_grid = dict(
    optimizer=optimizers,
    dropout_rate=dropout_rates,
    use_batch_norm=use_batch_norms,
    neurons=neurons,
    batch_size=batchSize,
    epochs=epochs,  )

model = KerasClassifier(build_fn=create_my_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)


<ipython-input-7-cff69d8d7669>:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_my_model, verbose=0)


In [8]:


grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

best_model = grid_result.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy with Best Hyperparameters:", test_accuracy)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.783317 using {'batch_size': 100, 'dropout_rate': 0.4, 'epochs': 200, 'neurons': 16, 'optimizer': 'SGD', 'use_batch_norm': True}
Test Accuracy with Best Hyperparameters: 0.7402597665786743
